In [1]:
import math 
import numpy as np 
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt 
import tensorflow as tf 
import keras 


In [2]:
df = pd.read_csv('AzureReadings_at_a_timestamp1.csv')  
df['timestamp'] =  pd.to_datetime(df['timestamp']) 
df = df.set_index('timestamp')
df.head() 

,norm_min_cpu,norm_max_cpu,norm_avg
timestamp,,,
2017-01-01 00:00:00,0.2283,0.2346,0.2314
2017-01-01 00:05:00,0.2023,0.2282,0.2152
2017-01-01 00:10:00,0.2120,0.2286,0.2203
2017-01-01 00:15:00,0.1809,0.2137,0.1973
2017-01-01 00:20:00,0.1807,0.2114,0.1960


In [3]:
dataset = df.values 
dataset = dataset.astype('float32') 

In [4]:
scaler = MinMaxScaler(feature_range=(0, 1)) 
dataset = scaler.fit_transform(dataset)

In [5]:
train_size = int(len(dataset) * 0.8) 
test_size = len(dataset) - train_size 
train, test = dataset[0:train_size, :], dataset[train_size:len(dataset), :]
print(len(train), len(test))

6912 1728


In [6]:
def create_training_dataset(dataset, look_back=1): 
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), :3]
        dataX.append(a)
        dataY.append(dataset[i + look_back, :])
    return np.array(dataX), np.array(dataY)

In [7]:
look_back = 5 
trainX, trainY = create_training_dataset(train, look_back=look_back)
testX, testY = create_training_dataset(test, look_back=look_back)

In [9]:
trainX.shape 

(6906, 5, 3)

In [10]:
model = keras.models.Sequential() 
model.add(keras.layers.LSTM(128, input_shape=(trainX.shape[1], trainX.shape[2])))
model.add(keras.layers.Dense(3))

In [11]:
adamOpt = tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, decay=0.0, amsgrad=False)
model.compile(loss='mean_squared_error', optimizer=adamOpt, metrics=['mae'])

C:\ProgramData\Anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               67584     
                                                                 
 dense (Dense)               (None, 3)                 387       
                                                                 
Total params: 67,971
Trainable params: 67,971
Non-trainable params: 0
_________________________________________________________________


In [13]:
# callback = keras.callbacks.EarlyStopping(monitor='loss', patience=3)
history = model.fit(trainX, trainY, validation_split=0.2,epochs=20, batch_size=64, verbose=2)

Epoch 1/20
87/87 - 5s - loss: 0.0044 - mae: 0.0414 - val_loss: 6.6758e-04 - val_mae: 0.0204 - 5s/epoch - 56ms/step
Epoch 2/20
87/87 - 1s - loss: 8.5123e-04 - mae: 0.0196 - val_loss: 6.1970e-04 - val_mae: 0.0198 - 856ms/epoch - 10ms/step
Epoch 3/20
87/87 - 1s - loss: 7.9726e-04 - mae: 0.0187 - val_loss: 5.7349e-04 - val_mae: 0.0187 - 845ms/epoch - 10ms/step
Epoch 4/20
87/87 - 1s - loss: 7.4873e-04 - mae: 0.0180 - val_loss: 5.4883e-04 - val_mae: 0.0182 - 836ms/epoch - 10ms/step
Epoch 5/20
87/87 - 1s - loss: 7.1328e-04 - mae: 0.0175 - val_loss: 5.3360e-04 - val_mae: 0.0180 - 834ms/epoch - 10ms/step
Epoch 6/20
87/87 - 1s - loss: 7.0376e-04 - mae: 0.0175 - val_loss: 5.6829e-04 - val_mae: 0.0186 - 833ms/epoch - 10ms/step
Epoch 7/20
87/87 - 1s - loss: 6.6191e-04 - mae: 0.0169 - val_loss: 5.0352e-04 - val_mae: 0.0175 - 831ms/epoch - 10ms/step
Epoch 8/20
87/87 - 1s - loss: 6.4337e-04 - mae: 0.0168 - val_loss: 5.7441e-04 - val_mae: 0.0191 - 831ms/epoch - 10ms/step
Epoch 9/20
87/87 - 1s - loss: 6

In [14]:
model.evaluate(testX, testY)

54/54 [==============================] - ETA: 0s - loss: 5.5713e-04 - mae: 0.016 - 0s 5ms/step - loss: 5.2860e-04 - mae: 0.0160


[0.0005285989609546959, 0.016033971682190895]

In [15]:
model.metrics_names

['loss', 'mae']

In [16]:
trainPredict = model.predict(trainX) # generate predictions for training
testPredict = model.predict(testX)

In [17]:
trainY = scaler.inverse_transform(trainY)
trainPredict = scaler.inverse_transform(trainPredict)
testY = scaler.inverse_transform(testY)
testPredict = scaler.inverse_transform(testPredict)

In [18]:
trainScore = math.sqrt(mean_squared_error(trainY[:], trainPredict[:])) # Estimate model performance
print('Train Score: %.2f RMSE' % (trainScore)
testScore = math.sqrt(mean_squared_error(testY[:], testPredict[:]))
print('Test Score: %.2f RMSE' % (testScore))


SyntaxError: invalid syntax (Temp/ipykernel_11480/2875214300.py, line 3)

In [ ]:
from sklearn.metrics import mean_absolute_error
trainScore = (mean_absolute_error(trainY[:], trainPredict[:]))
print('Train Score: %f MAE' % (trainScore))
testScore = math.sqrt(mean_absolute_error(testY[:], testPredict[:]))
print('Test Score: %f MAE' % (testScore))

In [ ]:
trainScore2 = np.mean(np.abs(trainPredict - trainY)/np.abs(trainY))
print('Train Score: %f MAPE' % (trainScore2))
testScore2 = np.mean(np.abs(testPredict - testY)/np.abs(testY))
print('Test Score: %f MAPE' % (testScore2))

In [ ]:
index=df.index
TestY= pd.DataFrame(testY,columns=['min_cpu','max_cpu','avg_cpu'])
PredY=pd.DataFrame(testPredict,columns=['min_cpu','max_cpu','avg_cpu'])

x=index[-1722:]
fig, axs = plt.subplots(3,figsize=(10,15)) # plot baseline and predictions

axs[0].plot(x,TestY.min_cpu,'.',label='Test min CPU',color='red')
axs[0].plot(x,PredY.min_cpu,'--',label='Predicted min CPU',color='black')
axs[0].legend()
axs[1].plot(x,TestY.max_cpu,'.',label='Test max CPU',color='magenta')
axs[1].plot(x,PredY.max_cpu,'--',label='Predicted max CPU',color='navy')
axs[1].legend()
axs[2].plot(x,TestY.avg_cpu,'.',label='Test avg CPU',color='orange')
axs[2].plot(x,PredY.avg_cpu,'--',label='Predicted avg CPU',color='darkgreen')
axs[2].legend()
for ax in axs.flat:
    ax.set(xlabel='Timestamp', ylabel='Workload (CPU)',autoscale_on=True)
for ax in axs:
    ax.label_outer()
#fig.suptitle('Prediction of Workload on Azure cloud at a particular timestamp',fontsize=20)
plt.savefig('gru_output.png', dpi = 300)
plt.show()